In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [4]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_3_Bin_Packing/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-21 15:31:56.586278


## Step 1 - Generate Mathematical Formulation 

In [5]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [6]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [7]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we can use a binary integer programming approach. Here are the parameters, decision variables, objective function, and constraints:

Parameters:
- Let I be the set of items, where |I| is the total number of items.
- Let B be the set of boxes, where |B| is the total number of boxes.
- Let w_i be the weight of item i, where i ∈ I.
- Let c_b be the capacity of box b, where b ∈ B.
- Let U be the upper bound on the number of boxes.

Decision Variables:
- Let x_ib be a binary variable indicating whether item i is packed into box b. x_ib = 1 if item i is packed into box b; otherwise, x_ib = 0.

Objective Function:
- The objective is to minimize the number of boxes used, which can be formulated as: min Σ b ∈ B y_b, where y_b is a binary variable indicating whether box b is used or not.

Constraints:
1. Each item is packed into exactly one box: Σ b∈B x_ib = 1, for all i ∈ I.
2. The total weight of items packed into a box does not exceed its capacity: Σ i∈I w_i * x_ib ≤ c_b * y_b, for all b ∈ B.
3. The number of used boxes does not exceed the upper bound: Σ b∈B y_b ≤ U.
4. The decision variables x_ib and y_b are binary: x_ib ∈ {0,1}, for all i ∈ I, b ∈ B, and y_b ∈ {0,1}, for all b ∈ B.

The objective function and constraints together form a mathematical optimization model for this problem. The goal is to find the optimal values of the decision variables that minimize the objective function while satisfying all the constraints.

In [8]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we can use a binary integer programming approach. Here are the parameters, decision variables, objective function, and constraints:

Parameters:
- Let I be the set of items, where |I| is the total number of items.
- Let B be the set of boxes, where |B| is the total number of boxes.
- Let w_i be the weight of item i, where i ∈ I.
- Let c_b be the capacity of box b, where b ∈ B.
- Let U be the upper bound on the number of boxes.

Decision Variables:
- Let x_ib be a binary variable indicating whether item i is packed into box b. x_ib = 1 if item i is packed into box b; otherwise, x_ib = 0.

Objective Function:
- The objective is to minimize the number of boxes used, which can be formulated as: min Σ b ∈ B y_b, where y_b is a binary variable indicating whether box b is used or not.

Constraints:
1. Each item is packed into exactly one box: Σ b∈B x_ib = 1, for all i ∈ I.
2. The total weight of items packed into a box does not 

## Step 2 - Generate the Pyomo Code

In [9]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [10]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [11]:
Markdown(response_2_text)

Here is a Python code using Pyomo to model and solve the given optimization problem. I have used sample data for the parameters `I`, `B`, `w_i`, `c_b`, and `U`. You can replace this sample data with your actual data as needed.

```python
from pyomo.environ import *

# Sample data
I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w_i = {'item1': 5, 'item2': 3, 'item3': 8}  # Weight of items
c_b = {'box1': 10, 'box2': 12, 'box3': 6}  # Capacity of boxes
U = 2  # Upper bound on the number of boxes

# Model
model = ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.B = Set(initialize=B)

# Parameters
model.w_i = Param(model.I, initialize=w_i)
model.c_b = Param(model.B, initialize=c_b)
model.U = Param(initialize=U)

# Decision variables
model.x_ib = Var(model.I, model.B, within=Binary)
model.y_b = Var(model.B, within=Binary)

# Objective function
def obj_rule(model):
    return sum(model.y_b[b] for b in model.B)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x_ib[i, b] for b in model.B) == 1

model.constraint1 = Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model, b):
    return sum(model.w_i[i] * model.x_ib[i, b] for i in model.I) <= model.c_b[b] * model.y_b[b]

model.constraint2 = Constraint(model.B, rule=constraint2_rule)

def constraint3_rule(model):
    return sum(model.y_b[b] for b in model.B) <= model.U

model.constraint3 = Constraint(rule=constraint3_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
for b in model.B:
    print(f"Box {b}: {'Used' if model.y_b[b]() > 0.5 else 'Not Used'}")
for i in model.I:
    for b in model.B:
        if model.x_ib[i, b]() > 0.5:
            print(f"Item {i} is packed in box {b}")
```

This code creates a binary integer programming model using Pyomo and uses the GLPK solver to find the optimal solution. You can replace the GLPK solver with other solvers as needed. The results are then printed out, showing the objective value, the boxes used, and the items packed in each box.

In [12]:
print(response_2_text)

Here is a Python code using Pyomo to model and solve the given optimization problem. I have used sample data for the parameters `I`, `B`, `w_i`, `c_b`, and `U`. You can replace this sample data with your actual data as needed.

```python
from pyomo.environ import *

# Sample data
I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w_i = {'item1': 5, 'item2': 3, 'item3': 8}  # Weight of items
c_b = {'box1': 10, 'box2': 12, 'box3': 6}  # Capacity of boxes
U = 2  # Upper bound on the number of boxes

# Model
model = ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.B = Set(initialize=B)

# Parameters
model.w_i = Param(model.I, initialize=w_i)
model.c_b = Param(model.B, initialize=c_b)
model.U = Param(initialize=U)

# Decision variables
model.x_ib = Var(model.I, model.B, within=Binary)
model.y_b = Var(model.B, within=Binary)

# Objective function
def obj_rule(model):
    return sum(model.y_b[b] for b in model.B)

model.obj = Objective(rule=o

### Code Executability

In [13]:
from pyomo.environ import *

# Sample data
I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w_i = {'item1': 5, 'item2': 3, 'item3': 8}  # Weight of items
c_b = {'box1': 10, 'box2': 12, 'box3': 6}  # Capacity of boxes
U = 2  # Upper bound on the number of boxes

# Model
model = ConcreteModel()

# Sets
model.I = Set(initialize=I)
model.B = Set(initialize=B)

# Parameters
model.w_i = Param(model.I, initialize=w_i)
model.c_b = Param(model.B, initialize=c_b)
model.U = Param(initialize=U)

# Decision variables
model.x_ib = Var(model.I, model.B, within=Binary)
model.y_b = Var(model.B, within=Binary)

# Objective function
def obj_rule(model):
    return sum(model.y_b[b] for b in model.B)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x_ib[i, b] for b in model.B) == 1

model.constraint1 = Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model, b):
    return sum(model.w_i[i] * model.x_ib[i, b] for i in model.I) <= model.c_b[b] * model.y_b[b]

model.constraint2 = Constraint(model.B, rule=constraint2_rule)

def constraint3_rule(model):
    return sum(model.y_b[b] for b in model.B) <= model.U

model.constraint3 = Constraint(rule=constraint3_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
for b in model.B:
    print(f"Box {b}: {'Used' if model.y_b[b]() > 0.5 else 'Not Used'}")
for i in model.I:
    for b in model.B:
        if model.x_ib[i, b]() > 0.5:
            print(f"Item {i} is packed in box {b}")

Objective value: 2.0
Box box1: Used
Box box2: Used
Box box3: Not Used
Item item1 is packed in box box1
Item item2 is packed in box box1
Item item3 is packed in box box2


### Solution Correctness

In [14]:
from pyomo.environ import *

# Sample data
I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w_i = {'item1': 5, 'item2': 3, 'item3': 8}  # Weight of items
c_b = {'box1': 10, 'box2': 12, 'box3': 6}  # Capacity of boxes
U = 2  # Upper bound on the number of boxes


# Bin capacity
bin_capacity = 9

# Weights of different types of items
weights = [2, 3, 4, 5, 6, 7, 8]

# Quantities of each type of item
quantities = [4, 2, 6, 6, 2, 2, 2]

# List of individual items
items = []
for j in range(len(weights)):
    for i in range(quantities[j]):
        items.append(weights[j])

# First-Fit Decreasing (FFD) Heuristic
def FFD(items, bin_capacity):
    remain = [bin_capacity]
    solution = [[]]
    for item in sorted(items, reverse=True):
        for j, free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                solution[j].append(item)
                break
        else:
            solution.append([item])
            remain.append(bin_capacity - item)
    return solution

# Upper bound on the number of bins
upper_bound_bins = len(FFD(items, bin_capacity))

# Sets and Parameters
num_items = len(items)
item_indices = list(range(num_items))
bin_indices = list(range(upper_bound_bins))
weights_dict = {i: v for i, v in enumerate(items)}
capacities_dict = {i: bin_capacity for i in range(upper_bound_bins)}
max_bins = upper_bound_bins

# Model
model = ConcreteModel()

model.I = Set(initialize=item_indices)
model.B = Set(initialize=bin_indices)
model.w_i = Param(model.I, initialize=weights_dict)
model.c_b = Param(model.B, initialize=capacities_dict)
model.U = Param(initialize=max_bins)


# Decision variables
model.x_ib = Var(model.I, model.B, within=Binary)
model.y_b = Var(model.B, within=Binary)

# Objective function
def obj_rule(model):
    return sum(model.y_b[b] for b in model.B)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Constraints
def constraint1_rule(model, i):
    return sum(model.x_ib[i, b] for b in model.B) == 1

model.constraint1 = Constraint(model.I, rule=constraint1_rule)

def constraint2_rule(model, b):
    return sum(model.w_i[i] * model.x_ib[i, b] for i in model.I) <= model.c_b[b] * model.y_b[b]

model.constraint2 = Constraint(model.B, rule=constraint2_rule)

def constraint3_rule(model):
    return sum(model.y_b[b] for b in model.B) <= model.U

model.constraint3 = Constraint(rule=constraint3_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
for b in model.B:
    print(f"Box {b}: {'Used' if model.y_b[b]() > 0.5 else 'Not Used'}")
for i in model.I:
    for b in model.B:
        if model.x_ib[i, b]() > 0.5:
            print(f"Item {i} is packed in box {b}")

Objective value: 13.0
Box 0: Used
Box 1: Used
Box 2: Used
Box 3: Used
Box 4: Used
Box 5: Used
Box 6: Used
Box 7: Used
Box 8: Used
Box 9: Used
Box 10: Used
Box 11: Used
Box 12: Used
Item 0 is packed in box 11
Item 1 is packed in box 9
Item 2 is packed in box 12
Item 3 is packed in box 12
Item 4 is packed in box 5
Item 5 is packed in box 2
Item 6 is packed in box 8
Item 7 is packed in box 0
Item 8 is packed in box 7
Item 9 is packed in box 6
Item 10 is packed in box 10
Item 11 is packed in box 12
Item 12 is packed in box 5
Item 13 is packed in box 8
Item 14 is packed in box 7
Item 15 is packed in box 0
Item 16 is packed in box 10
Item 17 is packed in box 6
Item 18 is packed in box 2
Item 19 is packed in box 3
Item 20 is packed in box 9
Item 21 is packed in box 11
Item 22 is packed in box 4
Item 23 is packed in box 1
